# USING TORCHEEG TO COMPLETE A DEEP LEARNING WORKFLOW

In this tutorial, we demonstrate a complete deep learning workflow using TorchEEG. We will cover the following aspects:

Utilizing Datasets and Neural Network architectures

Data Partitioning Strategies in TorchEEG

Leveraging Models and Trainers in TorchEEG

In [15]:
!py --version

Python 3.13.9


In [16]:
import torch, torch_scatter; print(torch.__version__, torch_scatter.__version__)

2.5.1+cu121 2.1.2+pt25cu121


In [17]:
!pip install torcheeg -q
!pip install ipywidgets -q

In [18]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu124.html -q

## Step 1: Initialize the Dataset

We use the DEAP dataset supported by TorchEEG. Each EEG sample is set to be 1 second long, encompassing 128 data points. The baseline signal is 3 seconds long, which we divide into three sections and then average to obtain the trial’s baseline signal.

During offline preprocessing, we divide each electrode’s EEG signal into 4 sub-bands, calculate the differential entropy for each sub-band as a feature, perform debaselining, and map onto a grid. Finally, the preprocessed EEG signals are saved locally. For online processing, we convert all EEG signals into Tensors, making them suitable for neural network input.

In [19]:
from torcheeg.datasets import DEAPDataset
from torcheeg import transforms

from torcheeg.datasets.constants.emotion_recognition.deap import \
    DEAP_CHANNEL_LOCATION_DICT


dataset = DEAPDataset(
    io_path = r".torcheeg/datasets_1769886804104_NjcHF", # cashed dataset path
    root_path=r"\deap-dataset\data_preprocessed_python",
     offline_transform=transforms.Compose([
        transforms.BandDifferentialEntropy(apply_to_baseline=True),
        transforms.ToGrid(DEAP_CHANNEL_LOCATION_DICT, apply_to_baseline=True)
    ]),
    online_transform=transforms.Compose(
        [transforms.BaselineRemoval(),
         transforms.ToTensor()]),
    label_transform=transforms.Compose([
        transforms.Select('valence'),
        transforms.Binary(5.0),
    ]))



[2026-02-01 22:31:22] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from .torcheeg/datasets_1769886804104_NjcHF.


In [20]:
eeg_tensor, label = dataset[0]
print(eeg_tensor.shape)

torch.Size([4, 9, 9])


In [21]:
from collections import Counter

counts = Counter(label for _, label in dataset)
zeros = counts[0]
ones = counts[1]

print(f"zeros: {zeros}")
print(f"ones: {ones}")

zeros: 59700
ones: 17100


In [22]:
from torcheeg.model_selection import train_test_split_groupby_trial

train_dataset, val_and_test_dataset = train_test_split_groupby_trial(
    dataset = dataset, test_size = 0.2, split_path = r".torcheeg/model_selection_1769971444157_smVDN"
)

val_dataset, test_dataset = train_test_split_groupby_trial(
    dataset = val_and_test_dataset, test_size = 0.5, split_path = r".torcheeg/model_selection_1769971447384_1ggwr"
)

[2026-02-01 22:31:25] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from .torcheeg/model_selection_1769971444157_smVDN.
[2026-02-01 22:31:25] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
[2026-02-01 22:31:25] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from .torcheeg/model_selection_1769971447384_1ggwr.
[2026-02-01 22:31:25] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.


In [33]:
!pip install torchinfo -q

In [29]:
import torch
torch.set_float32_matmul_precision("high")

In [ ]:
from torch.utils.data import DataLoader
from torcheeg.models import CCNN
from torcheeg.models import ViT
from torcheeg.trainers import ClassifierTrainer, WCELossTrainer
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

# import mlflow

# mlflow.pytorch.autolog()

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True, num_workers=12, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle = False, num_workers=12, pin_memory=True, persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False, num_workers=12, pin_memory=True, persistent_workers=True)


early_stop = EarlyStopping(
    monitor="val_f1score",   # or val_loss, val_accuracy, etc.
    mode="max",              # max for f1/accuracy, min for loss
    patience=10,             # epochs to wait
    min_delta=1e-4,          # minimum improvement
    verbose=True,
)


model = ViT(chunk_size=4,
            grid_size=(9, 9),
            t_patch_size=1,
            num_classes=2)

# model = CCNN(num_classes = 2, in_channels = 4, grid_size = (9, 9))

trainer = ClassifierTrainer(
    model=model,
    num_classes=2,
    lr=1e-4,
    weight_decay=1e-4,
    accelerator="gpu",
    metrics = ['precision', 'recall', 'f1score', 'accuracy', 'kappa'],
)

# trainer = WCELossTrainer(
#     model = model,
#     num_classes = 2,
#     class_frequency = [zeros, ones],
#     lr = 1e-4,
#     weight_decay = 1e-4,
#     accelerator = "gpu",
#     metrics = ['precision', 'recall', 'f1score', 'accuracy', 'kappa']
# )

trainer.fit(
    train_loader,
    val_loader,
    max_epochs = 50,
    # default_root_dir = f'./model',
    callbacks = [early_stop],
    enable_progress_bar = True,
    enable_model_summary = True,
)

trainer.test(
    test_loader,
    enable_progress_bar = True,
    enable_model_summary = True    
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode  | FLOPs
-------------------------------------------------------------------
0 | model         | ViT              | 113 K  | train | 0    
1 | ce_fn         | CrossEntropyLoss | 0      | train | 0    
2 | train_loss    | MeanMetric       | 0      | train | 0    
3 | val_loss      | MeanMetric       | 0      | train | 0    
4 | test_loss     | MeanMetric       | 0      | train | 0    
5 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[2026-02-02 00:49:03] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.897 val_precision: 0.094 val_recall: 0.500 val_f1score: 0.158 val_accuracy: 0.188 val_kappa: 0.000 


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

[2026-02-02 01:01:47] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.285 val_precision: 0.853 val_recall: 0.796 val_f1score: 0.819 val_accuracy: 0.885 val_kappa: 0.640 
[2026-02-02 01:01:47] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.232 train_precision: 0.878 train_recall: 0.833 train_f1score: 0.852 train_accuracy: 0.904 train_kappa: 0.705 
Metric val_f1score improved by 0.003 >= min_delta = 0.0001. New best score: 0.819
`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
LOCAL

Testing: |          | 0/? [00:00<?, ?it/s]

[2026-02-02 01:02:19] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.292 test_precision: 0.839 test_recall: 0.788 test_f1score: 0.809 test_accuracy: 0.877 test_kappa: 0.619 


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.8773437738418579
      test_f1score          0.8089087009429932
       test_kappa           0.6189537048339844
        test_loss           0.29179495573043823
     test_precision         0.8390719890594482
       test_recall          0.7877810001373291
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.29179495573043823,
  'test_precision': 0.8390719890594482,
  'test_recall': 0.7877810001373291,
  'test_f1score': 0.8089087009429932,
  'test_accuracy': 0.8773437738418579,
  'test_kappa': 0.6189537048339844}]

## Building RNN

In [43]:
import torch
from torch import nn

def conv(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 3, stride = 1, padding = 'same', bias = False)

def conv_down(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 3, stride = 3, bias = False)

def pool(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 3, stride = 3, bias = False)

def conv1x1(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 1, stride = 1, padding = 'same')

def bn(channels_in): 
    return nn.BatchNorm2d(channels_in)

def relu():
    return nn.ReLU()

def dropout(rate):
    return nn.Dropout(rate)




class Layer(nn.Sequential):
    def __init__(self, channels):
        super(Layer, self).__init__()
        self.add_module('norm', bn(channels))
        self.add_module('relu', relu())
        self.add_module('conv', conv(channels, channels))


class ResBlock(nn.Module):
    def __init__(self, channels, number_of_layers):
        super(ResBlock, self).__init__()
        for i in range(number_of_layers):
            layer = Layer(channels = channels)
            self.add_module('layer%d' % (i + 1), layer)
    
    def forward(self, input_data):
        features = input_data
        for _, layer in self.named_children():
            features = layer(features) + features            
        return features


class Transition_Layer(nn.Sequential):
    def __init__(self, channels_in, channels_out):
        super(Transition_Layer, self).__init__()
        self.add_module('norm', bn(channels_in))
        self.add_module('relu', nn.ReLU())
        self.add_module('conv1x1', conv1x1(channels_in, channels_out))
        self.add_module('pool', pool(channels_out, channels_out))
        # self.add_module('dropout', dropout(0.05))


class stage(nn.Sequential):
    def __init__(self, channels_in, channels_out, number_of_layers):
        super(stage, self).__init__()
        self.add_module('res_block', ResBlock(channels_in, number_of_layers))
        self.add_module('transition_layer', Transition_Layer(channels_in, channels_out))


class final_stage(nn.Sequential):
    def __init__(self, channels_in, channels_out, number_of_layers):
        super(final_stage, self).__init__()
        self.add_module('res_block', ResBlock(channels_in, number_of_layers))
        self.add_module('conv1x1', conv1x1(channels_in, channels_out))
        self.add_module('pool', conv_down(channels_in, channels_out))
        self.add_module('flatten', nn.Flatten())

class first_linear(nn.Sequential):
    def __init__(self, length_in, length_out):
        super(first_linear, self).__init__()
        self.add_module('linear', nn.Linear(length_in, length_out))
        self.add_module('norm', nn.BatchNorm1d(length_out))
        self.add_module('relu', relu())

class ResNet(nn.Sequential):
    def __init__(self):
        super(ResNet, self).__init__()
        # self.add_module('padding1', nn.ZeroPad2d(1)) # 9 -> 10
        # self.add_module('stage1', stage(4, 32, 3)) # 9 -> 3
        self.add_module('resBlock1', ResBlock(4, 3)) # 5 -> 5
        self.add_module('conv1x1_1', conv1x1(4, 32)) # 5 -> 5
        self.add_module('resBlock2', ResBlock(32, 2)) # 5 -> 5
        self.add_module('conv1x1_2', conv1x1(32, 64)) # 5 -> 5
        self.add_module('resBlock3', ResBlock(64, 2)) # 5 -> 5
        # self.add_module('resBlock1', ResBlock(32, 3)) # 5 -> 5
        # self.add_module('resBlock2', ResBlock(32, 3)) # 5 -> 5
        # self.add_module('padding2', nn.ZeroPad2d(1) ) # 5 -> 6
        # self.add_module('stage2', stage(32, 32, 3)) # 3 -> 1
        self.add_module('flatten', nn.Flatten()) # 1x1x32 = 288 
        self.add_module('linear1', first_linear(9*9*64, 2048))
        self.add_module('final_linear', nn.Linear(2048, 2))



        
        # # ===== Stage 1: low-level EEG features =====
        # self.add_module('res1_1', ResBlock(4, 3))        # depth at 4 channels
        # self.add_module('conv1', conv1x1(4, 32))         # 4 → 32 channels

        # self.add_module('res1_2', ResBlock(32, 3))       # depth at 32 channels
        # self.add_module('res1_3', ResBlock(32, 3))

        # # ===== Stage 2: mid-level patterns =====
        # self.add_module('conv2', conv1x1(32, 64))        # 32 → 64 channels

        # self.add_module('res2_1', ResBlock(64, 3))
        # self.add_module('res2_2', ResBlock(64, 3))
        # self.add_module('res2_3', ResBlock(64, 3))

        # # ===== Optional refinement stage =====
        # self.add_module('res3_1', ResBlock(64, 2))
        # self.add_module('res3_2', ResBlock(64, 2))

        # # ===== Head =====
        # self.add_module('flatten', nn.Flatten())
        # self.add_module('linear1', first_linear(9 * 9 * 64, 2048))
        # self.add_module('final_linear', nn.Linear(2048, 2))


In [44]:
from torchinfo import summary

model = ResNet()
summary(model, input_size=(64, 4, 9, 9))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [64, 2]                   --
├─ResBlock: 1-1                          [64, 4, 9, 9]             --
│    └─Layer: 2-1                        [64, 4, 9, 9]             --
│    │    └─BatchNorm2d: 3-1             [64, 4, 9, 9]             8
│    │    └─ReLU: 3-2                    [64, 4, 9, 9]             --
│    │    └─Conv2d: 3-3                  [64, 4, 9, 9]             144
│    └─Layer: 2-2                        [64, 4, 9, 9]             --
│    │    └─BatchNorm2d: 3-4             [64, 4, 9, 9]             8
│    │    └─ReLU: 3-5                    [64, 4, 9, 9]             --
│    │    └─Conv2d: 3-6                  [64, 4, 9, 9]             144
│    └─Layer: 2-3                        [64, 4, 9, 9]             --
│    │    └─BatchNorm2d: 3-7             [64, 4, 9, 9]             8
│    │    └─ReLU: 3-8                    [64, 4, 9, 9]             --
│    │    └─Conv

In [45]:
model = ResNet()

trainer = ClassifierTrainer(
    model=model,
    num_classes=2,
    lr=1e-4,
    weight_decay=1e-4,
    accelerator="gpu",
    metrics = ['precision', 'recall', 'f1score', 'accuracy', 'kappa'],
)

In [46]:
trainer.fit(
    train_loader,
    val_loader,
    max_epochs = 50,
    # default_root_dir = f'./model',
    callbacks = [early_stop],
    enable_progress_bar = True,
    enable_model_summary = True,
)

trainer.test(
    test_loader,
    enable_progress_bar = True,
    enable_model_summary = True    
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode  | FLOPs
-------------------------------------------------------------------
0 | model         | ResNet           | 10.7 M | train | 0    
1 | ce_fn         | CrossEntropyLoss | 0      | train | 0    
2 | train_loss    | MeanMetric       | 0      | train | 0    
3 | val_loss      | MeanMetric       | 0      | train | 0    
4 | test_loss     | MeanMetric       | 0      | train | 0    
5 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[2026-02-02 01:06:24] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.626 val_precision: 0.404 val_recall: 0.486 val_f1score: 0.441 val_accuracy: 0.789 val_kappa: -0.043 


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

[2026-02-02 01:14:34] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.162 val_precision: 0.933 val_recall: 0.933 val_f1score: 0.933 val_accuracy: 0.954 val_kappa: 0.866 
[2026-02-02 01:14:34] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.011 train_precision: 0.996 train_recall: 0.995 train_f1score: 0.995 train_accuracy: 0.997 train_kappa: 0.991 
Monitored metric val_f1score did not improve in the last 10 records. Best score: 0.946. Signaling Trainer to stop.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

[2026-02-02 01:15:05] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.160 test_precision: 0.932 test_recall: 0.933 test_f1score: 0.932 test_accuracy: 0.953 test_kappa: 0.865 


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy              0.953125
      test_f1score           0.932321310043335
       test_kappa           0.8646425604820251
        test_loss           0.15955571830272675
     test_precision         0.9321411848068237
       test_recall          0.9325016736984253
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.15955571830272675,
  'test_precision': 0.9321411848068237,
  'test_recall': 0.9325016736984253,
  'test_f1score': 0.932321310043335,
  'test_accuracy': 0.953125,
  'test_kappa': 0.8646425604820251}]